## Lecture 22-23 BIC
* **Define a function to compute BIC of given p and select the "optimal" p for AR(p) model**
    * The definition of BIC used in this lecture is taken from "_Introduction to Econometrics, 2nd edition, Pearson_"  P.551, equation 14.23.

Define a function named bicAR
* `lagmat2ds(x,maxlag0)`(statsmodels.tsa.tsatools.lagmat2ds): Generate lagmatrix for 2d array, columns arranged by variables(each column is a series of data starting from a certain lag). x is the original data. maxlag0 is the maximum lags to generate (including lag 0). See Example 1.
* `statsmodels.api.OLS()`: 
    * One of the arguments here is "missing".Available options are ‘none’, ‘drop’, and ‘raise’. If ‘none’, no nan checking is done. If ‘drop’, any observations with nans are dropped. If ‘raise’, an error is raised. Default is ‘none.’
    * statsmodels.api.OLS() also displays BIC, which can be extracted by ".bic" from results or simply read the summary table. See Example 2.
    * Here we need to include the constant term (estimation of $\mu$) in the regression by genrating a vector of 1s.

In [102]:
def bicAR(Y, pmax):
    """
    Calculate the Bayes Information Criterion for a univariate AR(p) model.
    
    Inputs:
        Y   : Times series data. 1 by n. A list or an array.
        pmax: User-specified upper bound of number of lags
        
    Outputs:
        BIC           : Bayes Information Criterion
        p_optimal     : The optimal p that generates BIC
        BIC_statsmodel: BIC given by statsmodel
        p_statsmodel  : The optimal p given by comparing BIC_statsmodel
    """
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.tsa.tsatools import lagmat2ds
    
    Y          = np.array(Y)    # in case the input is of other datatypes
    bic_aux    = np.zeros(pmax)
    T          = len(Y)    
    mu_aux     = np.transpose(np.matrix(np.ones(T)))
    bic_sm_aux = []
    
    for i_p in range(pmax): # 0,1,2..., pmax-1, so actual lag is i_p+1
        Ylag   = lagmat2ds(x=Y, maxlag0=i_p+1) # The first column is with lag 0   
        exogen = np.array(np.concatenate((mu_aux, Ylag[:,1:]), axis=1))

        for i in range(i_p+2):
            for j in range(T):
                if exogen[j,i] == 0:
                    exogen[j,i] = None 
                    
        reg1         = sm.OLS(endog=Y, exog=exogen, missing='drop')
        results      = reg1.fit()
        bic_sm_aux.append(results.bic)
        
        OLS_residual = results.resid    # a T-i_p-1 by 1 array; since we have drop those data without lags, here we take the whole array of residuals
        SSR          = np.sum(OLS_residual**2)
        bic_aux[i_p] = np.log(SSR/(T-i_p-1)) + (i_p+1+1)*np.log(T-i_p-1)/(T-i_p-1)
    
    BIC_statsmodel = min(bic_sm_aux)
    p_statsmodel   = bic_sm_aux.index(BIC_statsmodel)+1
    BIC            = np.nanmin(bic_aux)
    p_optimal      = np.where(bic_aux==BIC)[0][0]+1
    
    
    return BIC, p_optimal, BIC_statsmodel, p_statsmodel


**Example 1**
* Be careful: For time lags where there lack data, python replace them with 0, not None! So before we run the OLS regrssion, we must change those 0 values into None and then drop those datapoints in the regression.

In [103]:
import numpy as np
from statsmodels.tsa.tsatools import lagmat2ds
import pandas as pd

max_lag = 4

X = np.array([1,2,3,4,5,6,7])
Xlag = lagmat2ds(x=X, maxlag0=max_lag)
pd.DataFrame(Xlag, index = ['t=1','t=2','t=3','t=4','t=5','t=6','t=7'])

,0,1,2,3,4
t=1,1.0,0.0,0.0,0.0,0.0
t=2,2.0,1.0,0.0,0.0,0.0
t=3,3.0,2.0,1.0,0.0,0.0
t=4,4.0,3.0,2.0,1.0,0.0
t=5,5.0,4.0,3.0,2.0,1.0
t=6,6.0,5.0,4.0,3.0,2.0
t=7,7.0,6.0,5.0,4.0,3.0


* Convert 0 values into None

In [104]:
for i in range(max_lag+1):
    for j in range(len(X)):
        if Xlag[j,i] == 0:
            Xlag[j,i] = None
pd.DataFrame(Xlag, index = ['t=1','t=2','t=3','t=4','t=5','t=6','t=7'])

,0,1,2,3,4
t=1,1.0,NaN,NaN,NaN,NaN
t=2,2.0,1.0,NaN,NaN,NaN
t=3,3.0,2.0,1.0,NaN,NaN
t=4,4.0,3.0,2.0,1.0,NaN
t=5,5.0,4.0,3.0,2.0,1.0
t=6,6.0,5.0,4.0,3.0,2.0
t=7,7.0,6.0,5.0,4.0,3.0


**Invoke the bicAR function. Taking AR(3) as an example.**

* Set up parameters

In [105]:
class AR3_param:
    def __init__(self):
        self.phi1 = 0.5
        self.phi2 = 0.3
        self.phi3 = 0.1
        self.sigma2 = 2

AR3 = AR3_param()

In [106]:
print(AR3.phi1)
print(AR3.phi2)
print(AR3.phi3)
print(AR3.sigma2)

0.5
0.3
0.1
2


* Simulate data

In [154]:
import numpy as np
import statsmodels.api as sm

T = 10000
ar_param = np.array([1, -AR3.phi1, -AR3.phi2, -AR3.phi3])
ma_param = np.array([1])

np.random.seed(1) 
Y = sm.tsa.arma_generate_sample(ar=ar_param, ma=ma_param, nsample=T, sigma = AR3.sigma2**0.5)

* Select p using bicAR

In [155]:
from bic import bicAR
(BIC, p_optimal, BIC_statsmodel, p_statsmodel) = bicAR(Y,6)

p   = [p_optimal,p_statsmodel]
bic = [BIC, BIC_statsmodel]
import pandas as pd
pd.DataFrame({'selected p': p, 'BIC': bic}, index=['bicAR','statsmodel'])

,selected p,BIC
bicAR,3,0.693306
statsmodel,3,35311.476681


#### Difference between BIC obtained by bicAR and statsmodel
* The BIC that statsmodel applies is (approximately) the general form for any parametric models:  
(Wikipedia: https://en.wikipedia.org/wiki/Bayesian_information_criterion)  
$BIC = ln(n)k - 2ln\hat{L}$
    * $\hat{L}$: the maximized value of the likelihood function of the model.
    * $k$: the number of parameters estimated by the model. Note that this includes the intercept, the slope parameters, and the constant variance of the errors. **However, it seems that in statsmodel, the variance of errors does not account for the number of parameters.**
    * $n$: the number of observations.

 #### Derive BIC for linear regression models
 * $\left.Y\right|X \sim N(X\beta,\ \sigma^2I_n)$, where $Y$ is $n\times1$, $X$ is $n\times r$  
 
 
 * $\Rightarrow L(\beta\mid Y, X) = \frac{1}{\sqrt[]{2\pi}^{n}{\large \sigma^n}} {\Large e}^{-\frac{1}{2\sigma^2}(Y-X\beta)^\prime(Y-X\beta)}$  
 
 
 * In linear regression models, if all the basic assumptions are satisfied (exogeneity, homoscedastisity...), OLS estimatior is equivalent to ML estimator:  
 $\hat{\beta}_{\small ML}=(X^\prime X)^{-1}X^\prime Y$  
 
 $\hat{\sigma}^2=\frac{{\large (Y-X \hat{\beta})^\prime (Y-X \hat{\beta})}}{{\large n}}$
 
 
 * $\Rightarrow \hat{L}= \frac{1}{\sqrt[]{2\pi}^{n}{\large \hat{\sigma}^n}}{\Large e}^{-\frac{1}{2\hat{\sigma}^{2}}(\hat{\sigma}^{2} {\large n})}=\frac{1}{\sqrt[]{2\pi}^{n}{\large \hat{\sigma}^n}}{\Large e}^{-\frac{n}{2}}$  
 
    $\Rightarrow  ln\hat{L}= ln(\frac{1}{\sqrt[]{2\pi}^{n}})+ln(\frac{1}{\hat{\sigma}^{2}})^{\frac{n}{2}}-\frac{{\large n}}{2}= - \frac{{\large n}}{2}ln(2\pi)-\frac{{\large n}}{2}-\frac{{\large n}}{2}ln(\hat{\sigma}^{2})$  

* $\Rightarrow BIC \equiv ln(n)(r+1) + nln(2\pi) + n + nln(\hat{\sigma}^{2})$  
    
    $\qquad \quad = nln(\hat{\sigma}^{2}) + (r+1)ln(n) + nln(2\pi) + n$  
    
    $\qquad \quad = n\left( ln(\hat{\sigma}^{2})+ (r+1)\frac{{\large ln(n)}}{{\large n}}+ ln(2\pi)+1 \right)$


* Here in our $AR(p)$ setup:
    * $n = T-p$
    * $r+1 = p+1$: $p$ denotes number of lags (slopes in the linear regression model); intercept (constant $\mu$)  
    
    $\Rightarrow$ In statsmodel: $BIC_{AR}=(T-p)\left( ln(\hat{\sigma}^{2})+ (p+1)\frac{{\large ln(T-p)}}{{\large (T-p)}}+ ln(2\pi)+1 \right)$   $\ (Equation\ 1)$

    We can check this result in **Example 2** below.

* In the bicAR function, we use the formula in lecture notes which, in this case, is  
$BIC = ln(\hat{\sigma}^{2})+ (p+1)\frac{{\large ln(T-p)}}{{\large (T-p)}}$

* Let's convert BIC_statsmodel into BIC computed in bicAR

In [156]:
BIC_statsmodel/(T-p_statsmodel) -  np.log(2*np.pi)-1

0.6943302638547535

In [157]:
BIC # BIC computed in bicAR

0.6933061263014961

* **Example 2**: results of OLS regression (checking lag = 3)
    * Compare BIC given by Regression Results and that computed manually according to $Equation\ 1$

In [161]:
p = 3
T = len(Y)
mu_aux = np.transpose(np.matrix(np.ones(T)))
Ylag   = lagmat2ds(x=Y, maxlag0=p)
exogen = np.array(np.concatenate((mu_aux, Ylag[:,1:]), axis=1))

for i in range(p+1):
    for j in range(T):
        if exogen[j,i] == 0:
            exogen[j,i] = None 

reg0         = sm.OLS(endog=Y, exog=exogen, missing='drop')
sm_resid     = results.resid
ssr          = np.sum(sm_resid**2)
sm_bic       = results.bic
results      = reg0.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.731
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     9074.
Date:                Tue, 16 Apr 2019   Prob (F-statistic):               0.00
Time:                        01:38:32   Log-Likelihood:                -17637.
No. Observations:                9997   AIC:                         3.528e+04
Df Residuals:                    9993   BIC:                         3.531e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0135      0.014      0.954      0.3

In [162]:
BIC_manual = (T-p) * (np.log(ssr/(T-p))+ (p+1) * np.log(T-p)/(T-p)+ np.log(2*np.pi) + 1)

print(BIC_manual)

35311.4766806502


In [163]:
sm_bic

35311.476680650194